## Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

2025-02-27 08:32:56.932163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 08:32:56.946921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 08:32:56.951460: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 08:32:56.962835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


## Loading database

#### StandardScaler

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-27 08:33:07 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-27 08:33:07 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-27 08:33:07 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-27 08:33:07 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-27 08:33:07 [INFO]: Loaded successfully!
2025-02-27 08:33:20 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-27 08:33:20 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-27 08:33:21 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

#### MinMaxScaler

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-13 16:17:33 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-13 16:17:33 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-13 16:17:33 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-13 16:17:33 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-13 16:17:33 [INFO]: Loaded successfully!
2025-02-13 16:17:46 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-13 16:17:46 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-13 16:17:47 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler = physionet2012_dataset["scaler"]

In [4]:
def separating_dataset(dataset):

        dataset_for_training = {
            "X": dataset['train_X'],
        }

        dataset_for_validating = {
            "X": dataset['val_X'],
            "X_ori": dataset['val_X_ori']
        }

        dataset_for_testing_ori = {
            "X_ori": dataset['test_X_ori'],
            "female_gender_test_X_ori": dataset['female_gender_test_X_ori'],
            "male_gender_test_X_ori": dataset['male_gender_test_X_ori'],
            "undefined_gender_test_X_ori": dataset['undefined_gender_test_X_ori'],
            "more_than_or_equal_to_65_test_X_ori":  dataset['more_than_or_equal_to_65_test_X_ori'],
            "less_than_65_test_X_ori": dataset['less_than_65_test_X_ori'],
            "ICUType_1_test_X_ori": dataset['ICUType_1_test_X_ori'],
            "ICUType_2_test_X_ori": dataset['ICUType_2_test_X_ori'],
            "ICUType_3_test_X_ori": dataset['ICUType_3_test_X_ori'],
            "ICUType_4_test_X_ori": dataset['ICUType_4_test_X_ori'],
            "classificacao_undefined_test_X_ori": dataset['classificacao_undefined_test_X_ori'],
            "classificacao_baixo_peso_test_X_ori": dataset['classificacao_baixo_peso_test_X_ori'],
            "classificacao_normal_peso_test_X_ori": dataset['classificacao_normal_peso_test_X_ori'],
            "classificacao_sobrepeso_test_X_ori": dataset['classificacao_sobrepeso_test_X_ori'],
            "classificacao_obesidade_1_test_X_ori": dataset['classificacao_obesidade_1_test_X_ori'],
            "classificacao_obesidade_2_test_X_ori": dataset['classificacao_obesidade_2_test_X_ori'],
            "classificacao_obesidade_3_test_X_ori": dataset['classificacao_obesidade_3_test_X_ori']
        }

        dataset_for_testing = {
            "X": dataset['test_X'],
            "female_gender_test_X": dataset['female_gender_test_X'],
            "male_gender_test_X": dataset['male_gender_test_X'],
            "undefined_gender_test_X": dataset['undefined_gender_test_X'],
            "more_than_or_equal_to_65_test_X":  dataset['more_than_or_equal_to_65_test_X'],
            "less_than_65_test_X": dataset['less_than_65_test_X'],
            "ICUType_1_test_X": dataset['ICUType_1_test_X'],
            "ICUType_2_test_X": dataset['ICUType_2_test_X'],
            "ICUType_3_test_X": dataset['ICUType_3_test_X'],
            "ICUType_4_test_X": dataset['ICUType_4_test_X'],
            "classificacao_undefined_test_X": dataset['classificacao_undefined_test_X'],
            "classificacao_baixo_peso_test_X": dataset['classificacao_baixo_peso_test_X'],
            "classificacao_normal_peso_test_X": dataset['classificacao_normal_peso_test_X'],
            "classificacao_sobrepeso_test_X": dataset['classificacao_sobrepeso_test_X'],
            "classificacao_obesidade_1_test_X": dataset['classificacao_obesidade_1_test_X'],
            "classificacao_obesidade_2_test_X": dataset['classificacao_obesidade_2_test_X'],
            "classificacao_obesidade_3_test_X": dataset['classificacao_obesidade_3_test_X']
        }

        return dataset_for_training, dataset_for_validating, dataset_for_testing_ori, dataset_for_testing

In [5]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = separating_dataset(physionet2012_dataset)

In [26]:
## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori_standard.values(), dataset_for_testing_standard.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))# metric functions do not accpet input with NaNs, hence fill NaNs with 0


In [27]:
def pre_reshape(dataset):
    for i in range(len(dataset)):
        dataset[i] = dataset[i].reshape(len(dataset[i])*48, 37)
    return dataset

In [30]:
def reshape_variable(dataset):
        listaMed = []
        listaAux = []
        dataset_variable = []

        for i in range(len(dataset)):
            for j in range(37):
                for k in range(len(dataset[i])):
                    listaAux.append(dataset[i][k][j])
                listaMed.append(listaAux)
                listaAux = []
            listaMed = np.array(listaMed)
            dataset_variable.append(listaMed)
            listaMed = []

        return dataset_variable

In [28]:
test_X_indicating_mask = pre_reshape(test_X_indicating_mask)

In [31]:
test_X_indicating_mask = reshape_variable(test_X_indicating_mask)

In [33]:
test_X_ori = pre_reshape(test_X_ori)

In [35]:
test_X_ori = reshape_variable(test_X_ori)

In [36]:
teste = "teste"

In [7]:
test_X_indicating_mask_variable = []
test_X_ori_variable = []
for i in range(len(test_X_indicating_mask)):
    test_X_indicating_mask_variable.append(test_X_indicating_mask[i].reshape(37, len(test_X_indicating_mask[i]) * 48))
    test_X_ori_variable.append(test_X_ori[i].reshape(37, len(test_X_ori[i]) * 48))

In [8]:
test_X_ori_variable_ori = []
for i in dataset_for_testing_ori_standard.values():
    i = i.reshape(len(i)*48, 37)
    i = scaler.inverse_transform(i)
    test_X_ori_variable_ori.append(np.nan_to_num(i))

In [9]:
for i in range(len(test_X_ori_variable_ori)):
    test_X_ori_variable_ori[i] = test_X_ori_variable_ori[i].reshape(37, len(test_X_ori_variable_ori[i]))

## Inicialize the models

#### SAITS

In [10]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-27 08:33:57 [INFO]: No given device, using default device: cuda
2025-02-27 08:33:57 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250227_T083357
2025-02-27 08:33:57 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250227_T083357/tensorboard
2025-02-27 08:33:57 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [133]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-13 16:02:48 [INFO]: No given device, using default device: cuda
2025-02-13 16:02:48 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-13 16:02:48 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


#### BRITS

In [16]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:04:25 [INFO]: No given device, using default device: cuda
2025-02-12 21:04:25 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250212_T210425
2025-02-12 21:04:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250212_T210425/tensorboard
2025-02-12 21:04:25 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [22]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-13 15:15:27 [INFO]: No given device, using default device: cuda
2025-02-13 15:15:27 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-13 15:15:27 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


#### USGAN

In [29]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:23:25 [INFO]: No given device, using default device: cuda
2025-02-12 21:23:25 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250212_T212325
2025-02-12 21:23:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250212_T212325/tensorboard
2025-02-12 21:23:25 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [36]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-13 15:21:08 [INFO]: No given device, using default device: cuda
2025-02-13 15:21:08 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-13 15:21:08 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


#### GPVAE

In [31]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:42:23 [INFO]: No given device, using default device: cuda
2025-02-12 21:42:23 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250212_T214223
2025-02-12 21:42:23 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250212_T214223/tensorboard
2025-02-12 21:42:23 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [37]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-13 15:21:15 [INFO]: No given device, using default device: cuda
2025-02-13 15:21:15 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-13 15:21:15 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


#### MRNN

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 22:10:50 [INFO]: No given device, using default device: cuda
2025-02-12 22:10:50 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250212_T221050
2025-02-12 22:10:50 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250212_T221050/tensorboard
2025-02-12 22:10:50 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [115]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-13 15:37:48 [INFO]: No given device, using default device: cuda
2025-02-13 15:37:48 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-13 15:37:48 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


## Train/Load the models

#### SAITS

In [11]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-02-27 08:34:12 [INFO]: Epoch 001 - training loss: 0.7233, validation loss: 6.7652
2025-02-27 08:34:14 [INFO]: Epoch 002 - training loss: 0.5383, validation loss: 6.7408
2025-02-27 08:34:17 [INFO]: Epoch 003 - training loss: 0.4956, validation loss: 6.7218
2025-02-27 08:34:20 [INFO]: Epoch 004 - training loss: 0.4643, validation loss: 6.6987
2025-02-27 08:34:22 [INFO]: Epoch 005 - training loss: 0.4392, validation loss: 6.6806
2025-02-27 08:34:25 [INFO]: Epoch 006 - training loss: 0.4207, validation loss: 6.6686
2025-02-27 08:34:28 [INFO]: Epoch 007 - training loss: 0.4065, validation loss: 6.6527
2025-02-27 08:34:30 [INFO]: Epoch 008 - training loss: 0.3953, validation loss: 6.6548
2025-02-27 08:34:33 [INFO]: Epoch 009 - training loss: 0.3866, validation loss: 6.6511
2025-02-27 08:34:36 [INFO]: Epoch 010 - training loss: 0.3798, validation loss: 6.6584
2025-02-27 08:34:36 [INFO]: Finished training. The best model is from epoch#9.
2025-02-27 08:34:36 [INFO]: Saved the model to tuto

#### Load - Standard Scaler

In [75]:
saits.load("tutorial_results/imputation/saits/20250204_T160556/SAITS.pypots")

2025-02-13 15:33:34 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250204_T160556/SAITS.pypots


#### Load - Min/Max Scaler

In [136]:
saits.load("tutorial_results/imputation/saits/20250212_T210052/SAITS.pypots")

2025-02-13 16:05:05 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250212_T210052/SAITS.pypots


#### BRITS

In [17]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:06:06 [INFO]: Epoch 001 - training loss: 0.1748, validation loss: 0.0076
2025-02-12 21:06:06 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch1_loss0.007565443698937694.pypots
2025-02-12 21:06:57 [INFO]: Epoch 002 - training loss: 0.0984, validation loss: 0.0052
2025-02-12 21:06:57 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch2_loss0.005197163403499872.pypots
2025-02-12 21:07:49 [INFO]: Epoch 003 - training loss: 0.0821, validation loss: 0.0044
2025-02-12 21:07:49 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch3_loss0.0043778282163354255.pypots
2025-02-12 21:08:42 [INFO]: Epoch 004 - training loss: 0.0753, validation loss: 0.0038
2025-02-12 21:08:42 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch4_loss0.0038322627195157112.pypots
2025-02-12 21:09:34 [INFO]: Epoch 005 - training loss: 0.0707, validation loss

#### Load - Standard Scaler

In [23]:
brits.load("tutorial_results/imputation/brits/20250204_T162108/BRITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-1

#### Load - Min/Max Scaler

In [ ]:
brits.load("tutorial_results/imputation/brits/20250210_T210425/BRITS.pypots")

NameError: name 'brits' is not defined

#### USGAN

In [30]:
us_gan.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:25:15 [INFO]: Epoch 001 - generator training loss: -0.0597, discriminator training loss: 0.1872, validation loss: 0.0179
2025-02-12 21:25:15 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch1_loss0.01785113320996364.pypots
2025-02-12 21:26:37 [INFO]: Epoch 002 - generator training loss: -0.0020, discriminator training loss: 0.0545, validation loss: 0.0143
2025-02-12 21:26:37 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch2_loss0.014305492537096143.pypots
2025-02-12 21:28:01 [INFO]: Epoch 003 - generator training loss: 0.0047, discriminator training loss: 0.0385, validation loss: 0.0115
2025-02-12 21:28:01 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch3_loss0.011536479081648092.pypots
2025-02-12 21:29:26 [INFO]: Epoch 004 - generator training loss: 0.0015, discriminator training loss: 0.0321, validation loss: 0.0092
2025-02-12 21:29:26 [INFO]: Sav

#### Load - Standard Scaler

In [39]:
us_gan.load("tutorial_results/imputation/us_gan/20250205_T144707/USGAN.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-1

#### Load - Min/Max Scaler

In [ ]:
us_gan.load("tutorial_results/imputation/us_gan/20250205_T212325/USGAN.pypots")

#### GPVAE

In [32]:
gp_vae.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:42:40 [INFO]: Epoch 001 - training loss: 24477.7298, validation loss: 0.0211
2025-02-12 21:42:40 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch1_loss0.02106783300017317.pypots
2025-02-12 21:42:44 [INFO]: Epoch 002 - training loss: 22721.8937, validation loss: 0.0109
2025-02-12 21:42:44 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch2_loss0.010868297067160408.pypots
2025-02-12 21:42:47 [INFO]: Epoch 003 - training loss: 22712.9991, validation loss: 0.0108
2025-02-12 21:42:47 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch3_loss0.010810701572336257.pypots
2025-02-12 21:42:51 [INFO]: Epoch 004 - training loss: 22711.2009, validation loss: 0.0110
2025-02-12 21:42:55 [INFO]: Epoch 005 - training loss: 22710.5562, validation loss: 0.0109
2025-02-12 21:42:59 [INFO]: Epoch 006 - training loss: 22710.2269, validation loss: 0.0108
2025-02-12 21:42:59 [IN

#### Load - Standard Scaler

In [40]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE.pypots")

2025-02-13 15:21:34 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE.pypots


#### Load - MinMax Scaler

In [ ]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250205_T214223/GPVAE.pypots")

#### MRNN

In [11]:
mrnn.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 22:11:29 [INFO]: Epoch 001 - training loss: 0.1970, validation loss: 0.0284
2025-02-12 22:11:29 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250212_T221050/MRNN_epoch1_loss0.028413710246483485.pypots
2025-02-12 22:11:42 [INFO]: Epoch 002 - training loss: 0.0690, validation loss: 0.0341
2025-02-12 22:11:56 [INFO]: Epoch 003 - training loss: 0.0532, validation loss: 0.0426
2025-02-12 22:12:09 [INFO]: Epoch 004 - training loss: 0.0448, validation loss: 0.0483
2025-02-12 22:12:09 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-02-12 22:12:09 [INFO]: Finished training. The best model is from epoch#1.
2025-02-12 22:12:09 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250212_T221050/MRNN.pypots


#### Load - Standard Scaler

In [116]:
mrnn.load("tutorial_results/imputation/mrnn/20250212_T220555/MRNN.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-1

#### Load - MinMax Scaler

In [ ]:
mrnn.load("tutorial_results/imputation/mrnn/20250205_T221050/MRNN.pypots")

## Testing stage

#### SAITS

In [12]:
saits_imputation = []
for value in  dataset_for_testing_standard.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])


In [13]:
saits_imputation_variable = []
for i in range(len(saits_imputation)):
    saits_imputation_variable.append(saits_imputation[i].reshape(37, len(saits_imputation[i]) * 48))


In [14]:
saits_imputation_variable_ori = []
for i in range(len(saits_imputation_variable)):
    saits_imputation_variable_ori.append(saits_imputation_variable[i].reshape(len(saits_imputation_variable[i][0]), 37))
    saits_imputation_variable_ori[i] = scaler.inverse_transform(saits_imputation_variable_ori[i])
    saits_imputation_variable_ori[i] = saits_imputation_variable_ori[i].reshape(37, len(saits_imputation_variable_ori[i]))

#### BRITS

In [24]:
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [25]:
brits_imputation_variable = []
for i in range(len(brits_imputation)):
    brits_imputation_variable.append(brits_imputation[i].reshape(37, len(brits_imputation[i]) * 48))

In [26]:
brits_imputation_variable_ori = []
for i in range(len(brits_imputation_variable)):
    brits_imputation_variable_ori.append(brits_imputation_variable[i].reshape(len(brits_imputation_variable[i][0]), 37))
    brits_imputation_variable_ori[i] = scaler.inverse_transform(brits_imputation_variable_ori[i])
    brits_imputation_variable_ori[i] = brits_imputation_variable_ori[i].reshape(37, len(brits_imputation_variable_ori[i]))

#### USGAN

In [44]:
usgan_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   usgan_results = us_gan.predict(_dict)
   usgan_imputation.append(usgan_results["imputation"])

In [51]:
usgan_imputation_variable = []
for i in range(len(usgan_imputation)):
    usgan_imputation_variable.append(usgan_imputation[i].reshape(37, len(usgan_imputation[i]) * 48))

In [52]:
usgan_imputation_variable_ori = []
for i in range(len(usgan_imputation_variable)):
    usgan_imputation_variable_ori.append(usgan_imputation_variable[i].reshape(len(usgan_imputation_variable[i][0]), 37))
    usgan_imputation_variable_ori[i] = scaler.inverse_transform(usgan_imputation_variable_ori[i])
    usgan_imputation_variable_ori[i] = usgan_imputation_variable_ori[i].reshape(37, len(usgan_imputation_variable_ori[i]))

#### GPVAE

In [53]:
gpvae_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   gpvae_results = gp_vae.predict(_dict)
   gpvae_imputation.append(gpvae_results["imputation"])

In [54]:
gpvae_imputation_variable = []
for i in range(len(gpvae_imputation)):
    gpvae_imputation_variable.append(gpvae_imputation[i].reshape(37, len(gpvae_imputation[i]) * 48))

In [55]:
gpvae_imputation_variable_ori = []
for i in range(len(gpvae_imputation_variable)):
    gpvae_imputation_variable_ori.append(gpvae_imputation_variable[i].reshape(len(gpvae_imputation_variable[i][0]), 37))
    gpvae_imputation_variable_ori[i] = scaler.inverse_transform(gpvae_imputation_variable_ori[i])
    gpvae_imputation_variable_ori[i] = gpvae_imputation_variable_ori[i].reshape(37, len(gpvae_imputation_variable_ori[i]))

#### MRNN

In [117]:
mrnn_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   mrnn_results = mrnn.predict(_dict)
   mrnn_imputation.append(mrnn_results["imputation"])

In [118]:
mrnn_imputation_variable = []
for i in range(len(mrnn_imputation)):
    mrnn_imputation_variable.append(mrnn_imputation[i].reshape(37, len(mrnn_imputation[i]) * 48))

In [119]:
mrnn_imputation_variable_ori = []
for i in range(len(mrnn_imputation_variable)):
    mrnn_imputation_variable_ori.append(mrnn_imputation_variable[i].reshape(len(mrnn_imputation_variable[i][0]), 37))
    mrnn_imputation_variable_ori[i] = scaler.inverse_transform(mrnn_imputation_variable_ori[i])
    mrnn_imputation_variable_ori[i] = mrnn_imputation_variable_ori[i].reshape(37, len(mrnn_imputation_variable_ori[i]))

## Calculate mean absolute error

#### SAITS

In [15]:
testing_mae_saits_append_subgroups = []
testing_mae_saits_append_variables = []
for i in range(len(saits_imputation_variable)):
    for j in range(len(saits_imputation_variable[i])):
        testing_mae_saits_append_variables.append(calc_mae(saits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups.append(testing_mae_saits_append_variables)
    testing_mae_saits_append_variables = [] 

In [16]:
testing_mae_saits_append_subgroups_ori = []
testing_mae_saits_append_variables_ori = []
for i in range(len(saits_imputation_variable_ori)):
    for j in range(len(saits_imputation_variable_ori[i])):
        testing_mae_saits_append_variables_ori.append(calc_mae(saits_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups_ori.append(testing_mae_saits_append_variables_ori)
    testing_mae_saits_append_variables_ori = [] 

#### BRITS

In [61]:
testing_mae_brits_append_subgroups = []
testing_mae_brits_append_variables = []
for i in range(len(brits_imputation_variable)):
    for j in range(len(brits_imputation_variable[i])):
      testing_mae_brits_append_variables.append(calc_mae(brits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups.append(testing_mae_brits_append_variables)
    testing_mae_brits_append_variables = []

In [28]:
testing_mae_brits_append_subgroups_ori = []
testing_mae_brits_append_variables_ori = []
for i in range(len(brits_imputation_variable_ori)):
    for j in range(len(brits_imputation_variable_ori[i])):
        testing_mae_brits_append_variables_ori.append(calc_mae(brits_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups_ori.append(testing_mae_brits_append_variables_ori)
    testing_mae_brits_append_variables_ori = [] 

#### USGAN

In [62]:
testing_mae_usgan_append_subgroups = []
testing_mae_usgan_append_variables = []
for i in range(len(usgan_imputation_variable)):
    for j in range(len(usgan_imputation_variable[i])):
        testing_mae_usgan_append_variables.append(calc_mae(usgan_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_usgan_append_subgroups.append(testing_mae_usgan_append_variables)
    testing_mae_usgan_append_variables = []

In [63]:
testing_mae_usgan_append_subgroups_ori = []
testing_mae_usgan_append_variables_ori = []
for i in range(len(usgan_imputation_variable_ori)):
    for j in range(len(usgan_imputation_variable_ori[i])):
        testing_mae_usgan_append_variables_ori.append(calc_mae(usgan_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_usgan_append_subgroups_ori.append(testing_mae_usgan_append_variables_ori)
    testing_mae_usgan_append_variables_ori = [] 

#### GPVAE

In [64]:
testing_mae_gpvae_append_subgroups = []
testing_mae_gpvae_append_variables = []
for i in range(len(gpvae_imputation_variable)):
    for j in range(len(gpvae_imputation_variable[i])):
        testing_mae_gpvae_append_variables.append(calc_mae(gpvae_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_gpvae_append_subgroups.append(testing_mae_gpvae_append_variables)
    testing_mae_gpvae_append_variables = []

In [65]:
testing_mae_gpvae_append_subgroups_ori = []
testing_mae_gpvae_append_variables_ori = []
for i in range(len(gpvae_imputation_variable_ori)):
    for j in range(len(gpvae_imputation_variable_ori[i])):
        testing_mae_gpvae_append_variables_ori.append(calc_mae(gpvae_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_gpvae_append_subgroups_ori.append(testing_mae_gpvae_append_variables_ori)
    testing_mae_gpvae_append_variables_ori = [] 

#### MRNN

In [120]:
testing_mae_mrnn_append_subgroups = []
testing_mae_mrnn_append_variables = []
for i in range(len(mrnn_imputation_variable)):
    for j in range(len(mrnn_imputation_variable[i])):
        testing_mae_mrnn_append_variables.append(calc_mae(mrnn_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_mrnn_append_subgroups.append(testing_mae_mrnn_append_variables)
    testing_mae_mrnn_append_variables = []

In [121]:
testing_mae_mrnn_append_subgroups_ori = []
testing_mae_mrnn_append_variables_ori = []
for i in range(len(mrnn_imputation_variable_ori)):
    for j in range(len(mrnn_imputation_variable_ori[i])):
        testing_mae_mrnn_append_variables_ori.append(calc_mae(mrnn_imputation_variable_ori[i][j], test_X_ori_variable_ori[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_mrnn_append_subgroups_ori.append(testing_mae_mrnn_append_variables_ori)
    testing_mae_mrnn_append_variables_ori = [] 

## Results mean absolute error

In [17]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [18]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

In [19]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups[i][j])

SAITS - MAE
************
General
-------------
ALP : 0.24753747682772725
ALT : 0.22386561826443976
AST : 0.26390011033874394
Albumin : 0.30460072440659963
BUN : 0.2375681601417565
Bilirubin : 0.23380321850973695
Cholesterol : 0.2491483214437891
Creatinine : 0.26047016927572286
DiasABP : 0.2420778367737123
FiO2 : 0.24424449618037752
GCS : 0.27840230261451426
Glucose : 0.26653297163257905
HCO3 : 0.25182122979435745
HCT : 0.2414259153031803
HR : 0.23110145743659002
K : 0.24242084129869812
Lactate : 0.2581488483198929
MAP : 0.2585571852527747
MechVent : 0.25767516112043326
Mg : 0.2623976818304256
NIDiasABP : 0.23110403868897086
NIMAP : 0.23926897942239933
NISysABP : 0.2520714211895324
Na : 0.26171267756317185
PaCO2 : 0.2328809763031221
PaO2 : 0.24065223458917664
Platelets : 0.23545703209967894
RespRate : 0.25394173170099005
SaO2 : 0.2528067223207302
SysABP : 0.2373531593991957
Temp : 0.24228696492715965
TroponinI : 0.2422588041224756
TroponinT : 0.22885013770089938
Urine : 0.25354274614896

In [82]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups_ori[i][j])

SAITS - MAE
************
General
-------------
ALP : 9.769286317000859
ALT : 7.232163984410421
AST : 10.49980380848208
Albumin : 13.010662387965413
BUN : 8.31488287518348
Bilirubin : 8.169680217398982
Cholesterol : 12.814172763736101
Creatinine : 10.205049921129532
DiasABP : 10.002615766413816
FiO2 : 8.601420942411753
GCS : 11.4120014993566
Glucose : 9.50092465963547
HCO3 : 14.672736541354878
HCT : 9.151438121269212
HR : 8.443764395305111
K : 10.549347588170436
Lactate : 9.084613079742436
MAP : 9.668773775150866
MechVent : 10.196495717162325
Mg : 9.353224216906872
NIDiasABP : 13.367779366776432
NIMAP : 9.703498268592318
NISysABP : 14.66988598144468
Na : 12.684382902663724
PaCO2 : 7.629968925396834
PaO2 : 9.908180727888704
Platelets : 8.716901181443925
RespRate : 8.969988714454422
SaO2 : 9.966626215649663
SysABP : 11.798634887301546
Temp : 9.779241339801393
TroponinI : 10.543064326298298
TroponinT : 8.972393006409124
Urine : 10.039668578746483
WBC : 8.007987984641199
Weight : 9.94486734

In [17]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups[i][j])

SAITS - MAE
************
General
-------------
ALP : 0.031043257962657902
ALT : 0.028170038474726548
AST : 0.03196371920284396
Albumin : 0.03214313369916851
BUN : 0.029658829345883125
Bilirubin : 0.029296447671938196
Cholesterol : 0.03053975247482222
Creatinine : 0.031682666845023806
DiasABP : 0.027608199296117835
FiO2 : 0.029297089283156764
GCS : 0.03294213292290039
Glucose : 0.03134952347537376
HCO3 : 0.03186879309715356
HCT : 0.029033698823755963
HR : 0.028910497775058167
K : 0.028932415249945338
Lactate : 0.029528962856012284
MAP : 0.03083063732129137
MechVent : 0.02951229421593849
Mg : 0.033016768728603076
NIDiasABP : 0.02907365682914185
NIMAP : 0.02980853449343487
NISysABP : 0.030117783059762263
Na : 0.030114763411416532
PaCO2 : 0.028205603472151276
PaO2 : 0.0306448277088301
Platelets : 0.030168870919250548
RespRate : 0.030673072030405077
SaO2 : 0.03010541785488216
SysABP : 0.02965602938524825
Temp : 0.028850114467301468
TroponinI : 0.02876803508410699
TroponinT : 0.0277590267247

In [18]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups_ori[i][j])

SAITS - MAE
************
General
-------------
ALP : 18.486977223997854
ALT : 14.871245273878213
AST : 18.58145285801548
Albumin : 21.894383640778514
BUN : 13.666960210818639
Bilirubin : 14.44301429359136
Cholesterol : 20.55165680876346
Creatinine : 18.322628161393485
DiasABP : 17.731172582414192
FiO2 : 15.466098100351276
GCS : 18.58810738736534
Glucose : 16.093375106199186
HCO3 : 24.139360044976257
HCT : 16.711825152918077
HR : 17.201383657675294
K : 17.96171249895536
Lactate : 14.727792102519413
MAP : 16.180662175002254
MechVent : 16.99890072900802
Mg : 17.234027876624108
NIDiasABP : 22.400525634585897
NIMAP : 16.08105417543116
NISysABP : 25.264287104506103
Na : 20.48654368590296
PaCO2 : 14.248004577283192
PaO2 : 16.302670587445046
Platelets : 15.344959391495824
RespRate : 15.776237342984235
SaO2 : 18.349476889925022
SysABP : 20.497860551265042
Temp : 16.680592430277486
TroponinI : 17.029546154160492
TroponinT : 16.27680298513875
Urine : 16.04386478926532
WBC : 15.503169031091382
Wei

In [83]:
df_saits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_saits_mae[subgroups[i]] = testing_mae_saits_append_subgroups[i]

In [84]:
df_saits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.246029,0.220452,0.281318,0.024085,0.254108,0.286183,0.291996,0.220202,0.261112,0.264622,0.292983,0.277895,0.247686,0.237087,0.291813,0.180335,0.239991
1,ALT,0.220431,0.231447,0.222804,0.000000,0.254721,0.251318,0.278319,0.228516,0.228215,0.239234,0.248507,0.326892,0.210455,0.191214,0.277488,0.329962,0.228294
2,AST,0.262308,0.263395,0.237008,0.295356,0.248577,0.261737,0.274582,0.216824,0.246338,0.239527,0.266911,0.324861,0.244048,0.266117,0.213830,0.227629,0.178151
3,Albumin,0.302933,0.245631,0.308385,0.256960,0.225847,0.242713,0.218852,0.215799,0.271407,0.238350,0.241231,0.368676,0.253718,0.222560,0.287282,0.245201,0.181224
4,BUN,0.236598,0.224053,0.245069,0.038733,0.277838,0.235693,0.308771,0.204856,0.252615,0.247639,0.271124,0.281561,0.237951,0.256407,0.246900,0.205267,0.306612
5,Bilirubin,0.231291,0.250564,0.265393,0.098811,0.267808,0.258753,0.256349,0.205388,0.303023,0.255608,0.265551,0.239788,0.244810,0.222906,0.190586,0.242239,0.210118
6,Cholesterol,0.251680,0.231705,0.233628,0.169514,0.234425,0.274246,0.292118,0.201527,0.292166,0.258185,0.268551,0.178686,0.185244,0.216891,0.247719,0.277958,0.250856
7,Creatinine,0.255735,0.263072,0.283228,0.242628,0.258071,0.278139,0.214152,0.231790,0.269252,0.251534,0.281256,0.188572,0.222748,0.263322,0.231723,0.248706,0.189410
8,DiasABP,0.236598,0.265557,0.260386,0.129530,0.238133,0.369640,0.231949,0.207031,0.286422,0.259389,0.293405,0.250004,0.206029,0.202719,0.227379,0.176758,0.207900
9,FiO2,0.243917,0.263631,0.248451,0.118587,0.236096,0.279574,0.257063,0.218261,0.281995,0.237350,0.246835,0.178762,0.210799,0.248204,0.257538,0.231925,0.243773


In [85]:
df_saits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_saits_mae[subgroups[i]] = testing_mae_saits_append_subgroups_ori[i]

In [86]:
df_saits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,9.769286,7.200200,10.922719,0.113899,8.828034,8.117400,9.610810,8.095155,10.671866,12.057419,13.599245,13.521794,8.253710,9.774924,16.836756,7.143102,8.103314
1,ALT,7.232164,6.616699,7.316195,0.000000,6.750963,12.733098,12.281403,10.751182,6.578506,9.321097,7.945370,13.226596,6.392725,5.094062,10.760951,8.342766,6.884310
2,AST,10.499804,9.201366,9.575644,5.310749,6.953340,10.582048,11.088082,7.782618,10.607233,8.448042,11.005946,10.605885,6.028719,10.265188,8.526145,21.853399,10.383094
3,Albumin,13.010662,10.131554,12.188964,3.286186,10.022191,10.043459,10.277562,12.228630,9.704958,9.596813,9.143183,20.493394,9.171105,9.507930,25.683716,7.360320,6.116156
4,BUN,8.314883,7.605360,9.704161,0.222519,8.832360,10.597887,9.847439,7.578469,10.001864,10.703632,11.474351,5.701853,11.134026,9.818396,9.440903,12.548418,18.596328
5,Bilirubin,8.169680,9.227718,10.622080,0.163236,9.617512,16.071664,7.826642,8.492422,8.501961,8.296960,14.244621,9.517685,8.981639,8.188004,7.230170,10.800662,12.285537
6,Cholesterol,12.814173,7.521312,9.474109,2.167877,8.834195,11.620733,13.783223,7.643615,16.265023,9.974405,9.527539,5.613912,7.998073,7.939336,14.225323,8.444940,12.387430
7,Creatinine,10.205050,7.997739,12.672298,33.026899,8.623338,13.427512,7.156902,9.864978,9.079304,10.599511,12.672223,12.799531,9.823956,10.963991,11.666214,7.374244,6.863508
8,DiasABP,10.002616,8.225435,11.922314,0.518240,9.631190,20.647629,7.839593,9.860183,11.694537,10.272446,17.658567,14.126733,7.931998,6.737895,6.008000,4.609533,12.340421
9,FiO2,8.601421,10.389402,8.861236,1.771001,9.046276,9.775651,11.214373,11.160005,9.187407,9.692417,8.722582,8.487525,7.618852,12.318605,9.684853,6.607460,10.924719


Minimum MAE value in each subgroup

In [35]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
ALT
0.22043134772111558
--------------------
Female
NIDiasABP
0.21625072772560813
--------------------
Male
PaCO2
0.2108230852051344
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
PaO2
0.2052877660438494
--------------------
-65
RespRate
0.2195340976855963
--------------------
ICUType 1
Creatinine
0.21415215930457915
--------------------
ICUType 2
Ph
0.19352872186986245
--------------------
ICUType 3
PaCO2
0.2245388907288932
--------------------
ICUType 4
HCT
0.22102052107602238
--------------------
Undefined classification
TroponinT
0.22312300711184532
--------------------
Low Weight
K
0.1351692709542507
--------------------
Normal Weight
Cholesterol
0.1852442869645915
--------------------
Overweight
MAP
0.18507979199527627
--------------------
Obesity 1
Platelets
0.16087222354644645
--------------------
Obesity 2
WBC
0.1394584956765725
--------------------
Obesity 3
AST
0.17815062101412324
--------------------


Maximum MAE value in each subgroup

In [37]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")    

General
Albumin
0.30293279432474296
--------------------
Female
Urine
0.30370237839850694
--------------------
Male
Albumin
0.308384918688299
--------------------
Undefined Gender
HCT
0.5138421868725681
--------------------
+65
BUN
0.2778384531342736
--------------------
-65
DiasABP
0.3696399065245959
--------------------
ICUType 1
PaO2
0.3504441683775851
--------------------
ICUType 2
NIMAP
0.2556318391589281
--------------------
ICUType 3
PaO2
0.30687231283103145
--------------------
ICUType 4
MAP
0.3212451658407674
--------------------
Undefined classification
HCO3
0.29353148208906726
--------------------
Low Weight
NIMAP
0.6064582243175376
--------------------
Normal Weight
Glucose
0.2738133891557834
--------------------
Overweight
WBC
0.27807657394047125
--------------------
Obesity 1
HCO3
0.8932678493057511
--------------------
Obesity 2
SaO2
0.4265312900933624
--------------------
Obesity 3
NIMAP
0.4806281971702897
--------------------


#### BRITS

In [87]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups[i][j])

BRITS - MAE
************
General
-------------
ALP : 0.2707367647804506
ALT : 0.23896646826300247
AST : 0.28878421876356886
Albumin : 0.32158155068506383
BUN : 0.24372880799652813
Bilirubin : 0.2516354143011555
Cholesterol : 0.25355475512462006
Creatinine : 0.2723656167137088
DiasABP : 0.25076286871595893
FiO2 : 0.2514528318158146
GCS : 0.5404476843954903
Glucose : 0.2736264937761537
HCO3 : 0.2620078276782997
HCT : 0.24734390750479204
HR : 0.24368038771195344
K : 0.24904497537352557
Lactate : 0.2648993516274657
MAP : 0.27131795054600477
MechVent : 0.26718994902053
Mg : 0.2740963009645579
NIDiasABP : 0.23742945499040968
NIMAP : 0.24410922965602125
NISysABP : 0.26102740091037013
Na : 0.27076051164665826
PaCO2 : 0.24434648335436876
PaO2 : 0.261024932337629
Platelets : 0.24876741391001614
RespRate : 0.25789694312491024
SaO2 : 0.2707833075331891
SysABP : 0.25657190476225933
Temp : 0.2649548352687018
TroponinI : 0.2501403631064177
TroponinT : 0.2427855355938498
Urine : 0.2717610269676055
WBC

In [88]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups_ori[i][j])

BRITS - MAE
************
General
-------------
ALP : 10.71513517426209
ALT : 7.899825964689497
AST : 11.26177186637797
Albumin : 14.74343304442087
BUN : 7.848696142138587
Bilirubin : 8.552647934736108
Cholesterol : 12.64840876306235
Creatinine : 10.89831605304552
DiasABP : 10.944977976714137
FiO2 : 8.186566194244662
GCS : 13.84818972458662
Glucose : 9.994952176179163
HCO3 : 16.950263657534503
HCT : 9.731569553223181
HR : 8.910068588942758
K : 10.951118911456197
Lactate : 9.030069822880781
MAP : 9.629166115643441
MechVent : 10.764498340277948
Mg : 9.691140603240731
NIDiasABP : 14.32887311393969
NIMAP : 8.975985564108917
NISysABP : 17.276104435483482
Na : 13.270137043976131
PaCO2 : 8.031839675196174
PaO2 : 10.430397441383274
Platelets : 8.761709117045108
RespRate : 8.806987995822517
SaO2 : 11.305310310319907
SysABP : 13.693727316711549
Temp : 10.324551641837656
TroponinI : 11.15955958826452
TroponinT : 9.684630858426642
Urine : 10.863863539898626
WBC : 8.47939961095637
Weight : 11.356719

In [89]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups[i]

In [90]:
df_brits_mae 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.270737,0.256535,0.293297,0.329968,0.264696,0.315839,0.306754,0.226954,0.273958,0.271468,0.302521,0.256009,0.249952,0.258540,0.295754,0.216197,0.235523
1,ALT,0.238966,0.233629,0.244945,0.000000,0.276510,0.256375,0.292688,0.235882,0.250922,0.252148,0.281570,0.354939,0.216862,0.219696,0.282570,0.585253,0.224331
2,AST,0.288784,0.309354,0.251649,0.438452,0.269273,0.279676,0.316330,0.218515,0.275573,0.262117,0.281901,0.468715,0.247855,0.287826,0.233795,0.257504,0.206570
3,Albumin,0.321582,0.268955,0.321363,0.418167,0.259164,0.262846,0.263338,0.213691,0.289130,0.238066,0.247420,0.396643,0.267411,0.235706,0.330971,0.242369,0.202276
4,BUN,0.243729,0.233320,0.255147,0.086077,0.298458,0.258437,0.335850,0.245378,0.272088,0.261600,0.301877,0.375226,0.244099,0.266491,0.258967,0.220044,0.267429
5,Bilirubin,0.251635,0.272584,0.292882,0.139311,0.280527,0.276679,0.253002,0.211493,0.318044,0.274579,0.278961,0.242204,0.261701,0.247860,0.209038,0.249435,0.206693
6,Cholesterol,0.253555,0.240798,0.241246,0.173375,0.248621,0.283141,0.288232,0.211427,0.309410,0.267875,0.288611,0.168884,0.199758,0.235638,0.255247,0.314013,0.240939
7,Creatinine,0.272366,0.287540,0.288413,0.238730,0.266385,0.307470,0.229776,0.227727,0.292145,0.252393,0.275201,0.208665,0.256300,0.263794,0.254608,0.249333,0.203272
8,DiasABP,0.250763,0.271252,0.280030,0.090122,0.250686,0.368415,0.258635,0.211360,0.299583,0.270138,0.301323,0.297203,0.223460,0.210306,0.241157,0.223408,0.171138
9,FiO2,0.251453,0.262677,0.255937,0.073339,0.257094,0.287834,0.283073,0.226797,0.298260,0.250877,0.254612,0.181415,0.219981,0.232228,0.271426,0.270714,0.235543


In [91]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups_ori[i]

In [92]:
df_brits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,10.715135,8.501197,12.129913,1.560431,9.184957,9.448766,10.605360,9.240149,11.697007,12.061009,13.293554,13.289603,9.113300,9.876713,17.231677,6.339975,8.140584
1,ALT,7.899826,6.951257,7.294434,0.000000,7.542668,14.972953,12.062022,10.158185,7.269791,11.811162,8.111548,21.593454,6.598403,6.315828,10.667712,13.514883,7.516684
2,AST,11.261772,10.066463,11.732796,7.622379,7.248614,12.093161,11.547688,7.145767,10.586502,8.267928,11.544684,14.029931,6.069363,11.379993,7.943278,37.877976,9.980613
3,Albumin,14.743433,10.948176,15.748603,5.347832,11.627421,12.262826,11.625013,14.852903,10.321930,9.341758,9.684710,19.235285,9.276638,12.732945,38.197951,6.802864,6.858586
4,BUN,7.848696,7.909018,10.634979,1.163235,12.431516,10.800793,8.706540,8.867762,10.154652,11.720320,12.425427,6.260578,15.005418,9.729760,10.201111,23.037187,5.172259
5,Bilirubin,8.552648,8.792769,13.229901,0.333278,9.573323,19.163859,7.627356,10.759755,8.130710,9.643087,16.293699,9.433282,8.751381,8.680069,8.421324,10.084820,11.687593
6,Cholesterol,12.648409,7.525567,9.852638,2.217255,9.380563,11.399822,12.306403,8.092878,18.484953,10.873639,9.084614,6.756545,8.942141,8.070397,16.217997,11.447352,9.467242
7,Creatinine,10.898316,7.924208,12.972499,23.502145,8.677200,13.643019,7.534161,10.069408,11.207704,10.980717,12.244708,13.441368,10.877580,18.206448,11.692160,5.563947,6.776629
8,DiasABP,10.944978,10.123529,12.576162,0.360572,9.660426,22.915595,8.483409,17.009938,11.643033,9.674959,20.350309,17.486498,8.153454,7.104309,6.048146,5.018895,7.772816
9,FiO2,8.186566,12.325431,8.760452,0.598707,9.783900,9.241885,10.582603,11.332394,9.687451,9.299645,8.362476,5.969893,7.708252,11.337930,10.579355,6.806729,11.150927


Minimum MAE value in each subgroup

In [48]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.23742945499040968
--------------------
Female
NIDiasABP
0.2238358455137205
--------------------
Male
PaCO2
0.207682844444775
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
NIMAP
0.2160139074300404
--------------------
-65
RespRate
0.23431749476562846
--------------------
ICUType 1
Platelets
0.22327792083558695
--------------------
ICUType 2
Temp
0.19477174647491813
--------------------
ICUType 3
HCT
0.23571246018322012
--------------------
ICUType 4
HCT
0.22995585027402263
--------------------
Undefined classification
HCT
0.23881988273086038
--------------------
Low Weight
Urine
0.14774463446351938
--------------------
Normal Weight
Cholesterol
0.19975836468216204
--------------------
Overweight
MAP
0.18576449265978498
--------------------
Obesity 1
Mg
0.15740798042800078
--------------------
Obesity 2
WBC
0.15522959001932013
--------------------
Obesity 3
Lactate
0.17065355340995153
--------------------


Maximum MAE value in each subgroup

In [50]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
GCS
0.5404476843954903
--------------------
Female
Urine
0.3300605136654839
--------------------
Male
GCS
0.36500282476200896
--------------------
Undefined Gender
AST
0.43845240134512803
--------------------
+65
GCS
0.31723068628216455
--------------------
-65
DiasABP
0.36841543417646266
--------------------
ICUType 1
PaO2
0.3700832953509951
--------------------
ICUType 2
GCS
0.42147041589578793
--------------------
ICUType 3
Glucose
0.33969703536165907
--------------------
ICUType 4
Urine
0.3230442247268251
--------------------
Undefined classification
WBC
0.30855608846898475
--------------------
Low Weight
NIMAP
0.7751840826066394
--------------------
Normal Weight
GCS
0.4505264138330109
--------------------
Overweight
GCS
0.3038573484678385
--------------------
Obesity 1
HCO3
0.9005014291539718
--------------------
Obesity 2
ALT
0.5852528140030043
--------------------
Obesity 3
NIMAP
0.43202845565556747
--------------------


#### USGAN 

In [93]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups[i][j])

USGAN - MAE
************
General
-------------
ALP : 0.28327919700514537
ALT : 0.25698387822352164
AST : 0.29381425780834014
Albumin : 0.3316851965822937
BUN : 0.2558014721066809
Bilirubin : 0.26569659912151444
Cholesterol : 0.2717877965417484
Creatinine : 0.2807011303352554
DiasABP : 0.25648366397544786
FiO2 : 0.26940385736786227
GCS : 0.30802102369019674
Glucose : 0.27290957048290815
HCO3 : 0.2796737059846553
HCT : 0.2611459327041813
HR : 0.25871885034518405
K : 0.2614191212323757
Lactate : 0.2785402883573093
MAP : 0.28843045647664245
MechVent : 0.28180003190404546
Mg : 0.2834200527338646
NIDiasABP : 0.2516281229311397
NIMAP : 0.26122135945979064
NISysABP : 0.27605309628950486
Na : 0.27928460154237206
PaCO2 : 0.25462611017019066
PaO2 : 0.27273692419304074
Platelets : 0.2615611081737651
RespRate : 0.26835562858972456
SaO2 : 0.2824451172805901
SysABP : 0.2731640090539184
Temp : 0.27050709716394056
TroponinI : 0.2706214236991907
TroponinT : 0.259756080398352
Urine : 0.28173089277097757


In [94]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups_ori[i][j])

USGAN - MAE
************
General
-------------
ALP : 12.35599187996046
ALT : 8.54209455036235
AST : 13.054066539797418
Albumin : 14.179891938237494
BUN : 8.618095038293692
Bilirubin : 9.294435159603728
Cholesterol : 12.998335446870396
Creatinine : 11.865084637416281
DiasABP : 11.248977672202063
FiO2 : 8.89793082408602
GCS : 12.305456528942344
Glucose : 10.219146946337021
HCO3 : 14.946094305104268
HCT : 10.314829261309578
HR : 11.373623947693812
K : 11.83579452782401
Lactate : 9.425046351598308
MAP : 10.170724627986857
MechVent : 11.388369393696802
Mg : 10.628102239298093
NIDiasABP : 15.03112966893599
NIMAP : 10.491814940499225
NISysABP : 13.952622496556101
Na : 12.656286927674918
PaCO2 : 8.342248021835251
PaO2 : 11.1036736282715
Platelets : 9.249650850949573
RespRate : 9.21071728135099
SaO2 : 11.576195967710877
SysABP : 14.595332633831049
Temp : 11.945110780793822
TroponinI : 11.822928241659172
TroponinT : 9.633158420068131
Urine : 10.85618354427773
WBC : 9.359059939814312
Weight : 11.

In [100]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups[i]

In [101]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.283279,0.260204,0.292055,0.639126,0.278950,0.311200,0.324575,0.232860,0.284586,0.274382,0.308400,0.256726,0.278650,0.256221,0.302301,0.219735,0.282214
1,ALT,0.256984,0.266539,0.251540,0.000000,0.297289,0.277807,0.285709,0.251689,0.266937,0.261526,0.291585,0.301074,0.233951,0.228284,0.302541,0.448541,0.250768
2,AST,0.293814,0.303223,0.269565,0.384212,0.279343,0.280914,0.302549,0.236463,0.292150,0.263452,0.298456,0.373478,0.268835,0.295488,0.255971,0.239183,0.225336
3,Albumin,0.331685,0.281944,0.324433,0.283477,0.256423,0.277574,0.263155,0.215530,0.296938,0.245375,0.260768,0.352473,0.268574,0.237813,0.322414,0.232580,0.216323
4,BUN,0.255801,0.252679,0.267172,0.152919,0.299424,0.271188,0.310056,0.249563,0.286176,0.262668,0.302922,0.350310,0.263515,0.274829,0.271573,0.211423,0.304615
5,Bilirubin,0.265697,0.279324,0.293216,0.230283,0.285931,0.266058,0.283463,0.226065,0.332316,0.280361,0.293361,0.288386,0.269280,0.271721,0.254418,0.240187,0.236993
6,Cholesterol,0.271788,0.254231,0.258275,0.284804,0.255848,0.291156,0.310502,0.219850,0.307705,0.280798,0.290638,0.186191,0.214310,0.250294,0.257985,0.307338,0.283547
7,Creatinine,0.280701,0.296907,0.301210,0.221904,0.285033,0.296130,0.244120,0.244509,0.312245,0.275603,0.292766,0.186234,0.280157,0.285964,0.269462,0.279966,0.219875
8,DiasABP,0.256484,0.288802,0.280281,0.180854,0.264324,0.368516,0.293311,0.208421,0.310414,0.278863,0.313247,0.289131,0.223244,0.222269,0.259460,0.240750,0.203677
9,FiO2,0.269404,0.282501,0.275456,0.070404,0.265951,0.308677,0.299912,0.230760,0.305293,0.266495,0.265278,0.196520,0.244525,0.245101,0.278537,0.242282,0.225017


In [97]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups_ori[i]

In [98]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,12.355992,8.287835,13.655260,3.022444,10.443929,10.466376,11.102309,9.176101,13.922555,13.245725,14.000155,13.120227,12.126503,9.485620,17.286410,7.312059,9.248891
1,ALT,8.542095,7.565783,8.424811,0.000000,7.802415,15.260098,10.374043,10.747240,8.520809,15.878584,9.218647,27.850269,8.203513,6.318852,11.095401,12.787629,7.180419
2,AST,13.054067,11.630113,16.060514,6.237782,8.106599,12.019233,10.518553,8.605913,14.159352,8.066063,12.201314,14.238978,8.426175,13.272701,10.133774,19.115399,9.331968
3,Albumin,14.179892,13.854812,13.009964,3.625313,10.897244,12.925323,12.276105,9.699106,13.539229,9.986294,9.191124,18.574414,10.655238,12.606660,26.178724,7.616096,8.004924
4,BUN,8.618095,9.853193,10.729410,1.253390,10.856464,11.314423,8.758248,8.395070,13.260782,12.685381,13.832184,7.069753,13.226943,11.377676,9.866431,13.279722,10.520739
5,Bilirubin,9.294435,9.738397,10.547813,0.977116,9.747167,15.584304,9.037650,8.598986,10.229075,9.740014,15.403463,15.175580,8.237344,10.625534,10.980627,10.004611,15.649134
6,Cholesterol,12.998335,7.969230,12.385801,3.642288,9.425900,12.785014,13.306896,9.691137,17.852155,10.881608,9.846886,8.019840,11.714671,8.687923,12.806053,16.202108,14.381359
7,Creatinine,11.865085,8.946161,13.997118,28.451558,9.182559,13.380374,9.665395,9.671063,11.257829,11.767862,12.195590,10.615595,10.823183,12.817028,11.414230,8.013488,7.896098
8,DiasABP,11.248978,9.508887,12.363234,0.723583,9.496624,20.086925,12.369739,12.500518,12.141787,9.835656,16.268805,15.921786,7.321418,7.373341,6.727070,5.132018,9.598043
9,FiO2,8.897931,10.551741,9.333954,1.254852,9.504479,10.187516,10.669312,13.440051,11.388670,9.743399,8.835644,12.274672,12.124776,9.515141,12.091185,6.681561,8.329293


Minimum MAE value in each subgroup

In [102]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.2516281229311397
--------------------
Female
NIDiasABP
0.24216731176568626
--------------------
Male
PaCO2
0.23351034224264478
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
0.22837968883470952
--------------------
-65
RespRate
0.2510679315358316
--------------------
ICUType 1
Platelets
0.24184056448141877
--------------------
ICUType 2
DiasABP
0.2084214124015174
--------------------
ICUType 3
HCT
0.2527596471830482
--------------------
ICUType 4
Albumin
0.2453749441982267
--------------------
Undefined classification
HCT
0.24869123029565587
--------------------
Low Weight
Urine
0.1768420653889066
--------------------
Normal Weight
Cholesterol
0.21431008457946749
--------------------
Overweight
MAP
0.2195381558157749
--------------------
Obesity 1
Temp
0.187016339458737
--------------------
Obesity 2
WBC
0.16541041512975618
--------------------
Obesity 3
DiasABP
0.20367692515252409
--------------------


Maximum MAE value in each subgroup

In [43]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Mg
0.05296064324105656
--------------------
Female
MAP
0.05677509378339917
--------------------
Male
Bilirubin
0.05523572397582898
--------------------
Undefined Gender
DiasABP
0.12693270047493016
--------------------
+65
MechVent
0.057695967869961166
--------------------
-65
Glucose
0.056918280828891485
--------------------
ICUType 1
SysABP
0.09464095167411918
--------------------
ICUType 2
GCS
0.05075960704270495
--------------------
ICUType 3
HR
0.06392371835726103
--------------------
ICUType 4
SysABP
0.059539992399730506
--------------------
Undefined classification
MechVent
0.05941179715652995
--------------------
Low Weight
TroponinI
0.09469198543590841
--------------------
Normal Weight
Glucose
0.057020469869136926
--------------------
Overweight
Bilirubin
0.05662465584077266
--------------------
Obesity 1
Na
0.06700283247645569
--------------------
Obesity 2
TroponinI
0.08278429216698521
--------------------
Obesity 3
Cholesterol
0.07065558898885901
-------------------

#### GPVAE

In [103]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups[i][j])

GPVAE - MAE
************
General
-------------
ALP : 0.4636806691897626
ALT : 0.43988335188854844
AST : 0.49487440139032896
Albumin : 0.5215603429773343
BUN : 0.4266804258106264
Bilirubin : 0.4362346111747271
Cholesterol : 0.45224918270847075
Creatinine : 0.4757477721136869
DiasABP : 0.44289760298360664
FiO2 : 0.4433338364096275
GCS : 0.4799413889558506
Glucose : 0.4696212244440866
HCO3 : 0.44450931999694504
HCT : 0.4501893885953067
HR : 0.4400535016898482
K : 0.4529856488282846
Lactate : 0.4461148095715683
MAP : 0.46808219341677
MechVent : 0.4524354149233376
Mg : 0.4669271249641094
NIDiasABP : 0.4272968366403261
NIMAP : 0.4424829097324757
NISysABP : 0.4687402557776164
Na : 0.4649150142299674
PaCO2 : 0.4297125605908459
PaO2 : 0.4616128061306596
Platelets : 0.4373574331464247
RespRate : 0.45190520545070256
SaO2 : 0.46773353160229914
SysABP : 0.44711432869783224
Temp : 0.44845590548679254
TroponinI : 0.43925062163985207
TroponinT : 0.4395172281370459
Urine : 0.47446754863420765
WBC : 0.4

In [104]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups_ori[i][j])

GPVAE - MAE
************
General
-------------
ALP : 17.66386866392237
ALT : 12.898924792979548
AST : 18.033308533630034
Albumin : 20.12552717898322
BUN : 12.843901764136879
Bilirubin : 13.650733268857138
Cholesterol : 18.665341673185903
Creatinine : 16.232867470393124
DiasABP : 14.552670903385096
FiO2 : 13.888901250790205
GCS : 16.486643467420933
Glucose : 14.370490794554478
HCO3 : 22.138962510298153
HCT : 15.225100848448161
HR : 14.49028989533166
K : 16.448421793965252
Lactate : 13.832981364140569
MAP : 15.357806080294662
MechVent : 16.107622993237168
Mg : 14.64547112995032
NIDiasABP : 20.29543083869573
NIMAP : 14.721912643772923
NISysABP : 24.741573347631547
Na : 19.2346623015149
PaCO2 : 12.44587846519099
PaO2 : 16.039174769584378
Platelets : 13.908819015384314
RespRate : 13.984318392588822
SaO2 : 17.16442693307015
SysABP : 20.08565551411571
Temp : 15.744488397654182
TroponinI : 15.004118583326601
TroponinT : 14.565481353920214
Urine : 16.307476031882597
WBC : 13.641587981168835
Wei

In [105]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups[i]

In [106]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.463681,0.437348,0.481185,0.188923,0.452031,0.500722,0.498392,0.398523,0.468410,0.453942,0.492038,0.401444,0.439143,0.440158,0.454013,0.354501,0.441009
1,ALT,0.439883,0.452353,0.424574,0.000000,0.491392,0.458706,0.486426,0.395146,0.451703,0.457088,0.503753,0.692737,0.422994,0.400253,0.492085,0.686335,0.415788
2,AST,0.494874,0.506453,0.450780,0.724645,0.482102,0.467843,0.510333,0.402633,0.515917,0.438254,0.490237,0.607884,0.429936,0.489471,0.417430,0.416185,0.333943
3,Albumin,0.521560,0.468480,0.511329,1.001637,0.443553,0.456627,0.458601,0.362322,0.501570,0.438250,0.441548,0.548004,0.448468,0.442848,0.500192,0.440872,0.377416
4,BUN,0.426680,0.446517,0.412046,0.150895,0.471250,0.431017,0.474564,0.397925,0.453885,0.427392,0.470006,0.563741,0.465482,0.483913,0.430060,0.423617,0.591936
5,Bilirubin,0.436235,0.479521,0.494864,0.734586,0.456773,0.467347,0.458350,0.400101,0.519027,0.488478,0.461902,0.496981,0.403788,0.446324,0.393508,0.463751,0.395469
6,Cholesterol,0.452249,0.428988,0.455132,0.145363,0.453071,0.474963,0.485967,0.381231,0.520121,0.471640,0.496510,0.305876,0.352159,0.400880,0.462925,0.515067,0.427294
7,Creatinine,0.475748,0.492619,0.491877,0.501357,0.436339,0.506338,0.455648,0.404820,0.508868,0.468376,0.486859,0.408171,0.432863,0.438505,0.453152,0.485521,0.450144
8,DiasABP,0.442898,0.462887,0.478513,0.051857,0.460529,0.553068,0.417038,0.388051,0.516924,0.470249,0.519733,0.484796,0.390566,0.396541,0.395753,0.448396,0.352941
9,FiO2,0.443334,0.478511,0.435290,0.350734,0.450360,0.482558,0.455084,0.370986,0.521568,0.461136,0.454381,0.324044,0.388751,0.413061,0.407168,0.443543,0.378275


In [107]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups_ori[i]

In [108]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,17.663869,14.119615,18.597227,0.893423,14.541637,16.467419,15.955826,14.214014,19.168067,18.216104,17.353751,17.558109,19.568166,16.027809,21.211593,9.113255,13.631775
1,ALT,12.898925,12.869001,12.734366,0.000000,12.857347,25.231621,16.602458,15.848211,12.390530,20.665908,13.319186,56.732745,11.488265,10.821498,16.267777,18.268191,14.108837
2,AST,18.033309,17.628276,18.385392,14.229827,12.512492,16.923227,16.619766,13.249340,17.940974,12.893960,16.027415,18.544132,10.870046,17.945833,10.894314,46.771728,14.957980
3,Albumin,20.125527,16.858577,19.705958,12.809677,16.263546,17.629957,17.989338,20.899929,17.820365,14.984511,14.373039,25.337974,13.534485,21.964610,43.704470,9.865847,8.841502
4,BUN,12.843902,13.621606,15.088762,2.245535,18.457955,15.109284,14.201617,11.592660,15.809395,18.492304,17.859388,10.096330,22.636110,16.983376,14.753277,33.952205,22.599530
5,Bilirubin,13.650733,16.416270,19.433722,7.921325,14.858681,26.502625,11.571318,15.848804,13.196752,14.270541,22.709318,13.888710,12.107942,15.997882,12.915547,20.113805,23.591540
6,Cholesterol,18.665342,12.984090,15.992524,1.859016,14.849560,18.296629,20.710092,11.940921,25.210761,15.895168,15.903612,8.622644,12.919128,12.124306,21.923270,23.780905,23.526694
7,Creatinine,16.232867,14.718180,17.968384,23.202869,13.103127,20.321575,14.998470,14.627154,17.220862,16.471640,17.604799,18.423278,15.675790,24.844370,17.921555,11.791631,12.888248
8,DiasABP,14.552671,14.489022,19.211604,0.207476,14.603501,27.399853,13.420351,21.109982,17.377013,16.482854,26.753407,21.776472,13.072125,10.806056,9.826246,9.762096,14.383420
9,FiO2,13.888901,17.902364,13.128238,6.029407,15.567941,13.743428,13.904793,15.196091,14.984778,14.721499,12.951522,16.639314,14.988416,15.933269,12.687322,12.103499,15.058894


Minimum MAE value in each subgroup

In [47]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
PaCO2
0.056608179708824445
--------------------
Female
ALP
0.05943705556438818
--------------------
Male
PaCO2
0.05667613500395072
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
0.05565149489069709
--------------------
-65
Mg
0.059004120155529874
--------------------
ICUType 1
Platelets
0.05510620406104771
--------------------
ICUType 2
Mg
0.0488913377924879
--------------------
ICUType 3
PaCO2
0.05234645843986417
--------------------
ICUType 4
Cholesterol
0.05770430003196169
--------------------
Undefined classification
NIDiasABP
0.05919388716671923
--------------------
Low Weight
Platelets
0.041563009396780023
--------------------
Normal Weight
Mg
0.05085602784017245
--------------------
Overweight
PaCO2
0.047466605986953805
--------------------
Obesity 1
Bilirubin
0.04580181353821788
--------------------
Obesity 2
WBC
0.04208152342192024
--------------------
Obesity 3
FiO2
0.046362065784707064
--------------------


Maximum MAE value in each subgroup

In [48]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
AST
0.06931117774731774
--------------------
Female
FiO2
0.071832550791953
--------------------
Male
Creatinine
0.06965518953887552
--------------------
Undefined Gender
DiasABP
0.21033088366169542
--------------------
+65
BUN
0.06901402131591192
--------------------
-65
TroponinI
0.07859482135681735
--------------------
ICUType 1
PaO2
0.08124137487348888
--------------------
ICUType 2
Lactate
0.06164341253363296
--------------------
ICUType 3
TroponinI
0.07998017140527885
--------------------
ICUType 4
TroponinI
0.07497044203952626
--------------------
Undefined classification
GCS
0.07334575895254153
--------------------
Low Weight
TroponinT
0.10842176540684564
--------------------
Normal Weight
PaO2
0.07129215410887764
--------------------
Overweight
Mg
0.06900237369132736
--------------------
Obesity 1
PaCO2
0.07076721832273784
--------------------
Obesity 2
Na
0.10751933736913757
--------------------
Obesity 3
GCS
0.08768001772404654
--------------------


#### MRNN

In [122]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups[i][j])

MRNN - MAE
************
General
-------------
ALP : 0.6891042237096028
ALT : 0.6697312359094596
AST : 0.7416927518816129
Albumin : 0.7463954870157028
BUN : 0.6355481879591666
Bilirubin : 0.6416969716355293
Cholesterol : 0.6727495200279108
Creatinine : 0.6716671861362805
DiasABP : 0.6808283782616357
FiO2 : 0.6227840389728461
GCS : 0.6692272659201319
Glucose : 0.6720891461117408
HCO3 : 0.6589864147836295
HCT : 0.6458595741032642
HR : 0.6621585497680288
K : 0.6896896911599226
Lactate : 0.6430019270508706
MAP : 0.6848244816182206
MechVent : 0.6811896916910888
Mg : 0.650124389963784
NIDiasABP : 0.6215348600145024
NIMAP : 0.6550581111501786
NISysABP : 0.6952255969268104
Na : 0.6844226542870258
PaCO2 : 0.6287560867465835
PaO2 : 0.6655651204171794
Platelets : 0.6457055708189681
RespRate : 0.6806955443585905
SaO2 : 0.6720348938943402
SysABP : 0.7056722589941172
Temp : 0.680304816992715
TroponinI : 0.6713200044286652
TroponinT : 0.6750343726015907
Urine : 0.7106147217278526
WBC : 0.6661850898761

In [123]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups_ori[i][j])

MRNN - MAE
************
General
-------------
ALP : 24.516062374665196
ALT : 19.76140632793386
AST : 25.334944491362485
Albumin : 26.331253385358107
BUN : 19.004619375582397
Bilirubin : 18.452783496681125
Cholesterol : 26.299393281792913
Creatinine : 22.969102945048505
DiasABP : 22.819474031836187
FiO2 : 18.642404975239664
GCS : 23.672211796391895
Glucose : 20.773787921510856
HCO3 : 28.37746923931187
HCT : 21.94089499135492
HR : 22.83486947107638
K : 24.042471107344394
Lactate : 19.56257335052851
MAP : 22.254789915310006
MechVent : 23.47414708845794
Mg : 21.321667375671684
NIDiasABP : 26.760455598762196
NIMAP : 21.79631217956344
NISysABP : 31.151780132036247
Na : 26.099145682984634
PaCO2 : 18.550951472490553
PaO2 : 21.59285145797797
Platelets : 20.77311479593673
RespRate : 21.083665620517948
SaO2 : 24.205175390018802
SysABP : 27.604479803721574
Temp : 23.031004926304398
TroponinI : 21.541083986786916
TroponinT : 20.447983641733366
Urine : 22.28906228159628
WBC : 21.109446916738502
Weig

In [124]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups[i]

In [125]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.689104,0.667335,0.696357,0.507509,0.646565,0.687002,0.672778,0.620173,0.665549,0.670807,0.726986,0.701119,0.628677,0.624297,0.645220,0.575216,0.737773
1,ALT,0.669731,0.653103,0.654165,0.000000,0.725275,0.686937,0.749737,0.582993,0.685512,0.722545,0.730973,0.722608,0.655463,0.628625,0.628376,0.931739,0.845520
2,AST,0.741693,0.756573,0.673059,0.083308,0.705751,0.681755,0.695791,0.634816,0.798241,0.634615,0.712305,0.859331,0.675043,0.669676,0.591304,0.582306,0.681127
3,Albumin,0.746395,0.704606,0.727019,1.014342,0.673047,0.653678,0.674022,0.541006,0.717779,0.665241,0.684194,0.676810,0.665176,0.608718,0.683893,0.600455,0.590899
4,BUN,0.635548,0.699937,0.587745,0.475151,0.725550,0.627522,0.714661,0.647397,0.701698,0.619873,0.671836,0.772429,0.699477,0.665936,0.619232,0.630307,0.729258
5,Bilirubin,0.641697,0.700051,0.677235,0.408022,0.676139,0.655981,0.633770,0.604834,0.775192,0.679092,0.706135,0.533569,0.639532,0.638812,0.480346,0.669950,0.587823
6,Cholesterol,0.672750,0.669457,0.653943,0.269789,0.681913,0.652739,0.737699,0.605656,0.750578,0.646315,0.678274,0.646438,0.625275,0.602681,0.651379,0.839308,0.649382
7,Creatinine,0.671667,0.656672,0.728552,0.425493,0.649219,0.734455,0.650856,0.618545,0.738558,0.676408,0.698230,0.642483,0.664061,0.599628,0.655849,0.603157,0.854659
8,DiasABP,0.680828,0.680967,0.678367,0.561185,0.643803,0.815644,0.633256,0.588678,0.741413,0.635030,0.744099,0.770286,0.596444,0.554561,0.598039,0.468811,0.654385
9,FiO2,0.622784,0.697946,0.599732,0.278597,0.640812,0.669459,0.712953,0.560267,0.748908,0.682014,0.674755,0.651656,0.583494,0.645466,0.603720,0.625158,0.472036


In [126]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups_ori[i]

In [127]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,24.516062,20.988576,25.161750,2.400024,21.647512,21.342903,21.623159,22.556301,24.643770,25.235900,24.964435,23.224034,23.903568,22.415625,26.369533,18.067072,21.213121
1,ALT,19.761406,18.952402,19.638461,0.000000,19.334505,28.594732,21.537138,21.895695,20.141416,26.413404,20.281471,49.282872,18.282165,17.499217,21.309279,23.846820,26.222886
2,AST,25.334944,26.579326,23.612418,1.721962,20.749387,22.681041,21.601566,22.529802,27.849808,17.648277,23.625636,30.477203,16.838297,21.969123,22.743730,42.323570,26.307154
3,Albumin,26.331253,24.234270,26.431416,12.972168,22.817826,23.952954,23.671935,25.987292,25.237095,21.801892,21.890839,21.766353,19.732294,23.472464,46.049314,14.422156,13.395204
4,BUN,19.004619,22.210606,20.262097,8.086815,25.856101,22.605216,22.017617,19.963648,23.061506,22.986402,24.051789,14.019030,30.503317,23.307691,22.347369,35.352641,24.026583
5,Bilirubin,18.452783,22.923556,25.911022,1.685001,21.694257,30.051792,14.924928,20.757113,20.820588,20.402875,28.372613,17.800845,18.906356,18.891256,18.011494,33.193840,26.730912
6,Cholesterol,26.299393,19.731602,24.238783,3.450256,22.123994,24.041822,26.076719,19.005683,31.778662,21.044500,22.223414,19.980726,22.072709,20.010585,28.456079,29.843592,34.492058
7,Creatinine,22.969103,19.777166,25.650668,7.000729,20.222336,26.319398,22.241140,21.736267,24.263815,22.200094,23.620616,19.969087,22.416242,27.518229,26.033101,16.575496,24.404238
8,DiasABP,22.819474,20.141272,24.916647,2.245258,20.356127,37.178290,21.081126,27.794043,23.536936,20.348501,34.895308,34.136425,18.459370,16.313008,15.789321,10.378335,23.271778
9,FiO2,18.642405,24.151652,18.766815,4.311239,21.771360,18.807980,22.607375,22.576708,22.605700,21.172092,19.557118,30.204604,20.298861,22.836200,18.404152,17.528061,18.111971


Minimum MAE value in each subgroup

In [56]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.6215348600145024
--------------------
Female
TroponinT
0.62921742531125
--------------------
Male
PaCO2
0.5871804208287293
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
0.5893284527545951
--------------------
-65
NIDiasABP
0.6003298402934673
--------------------
ICUType 1
Temp
0.6057869251152296
--------------------
ICUType 2
MAP
0.5319993160382074
--------------------
ICUType 3
PaCO2
0.6184535399366906
--------------------
ICUType 4
NIDiasABP
0.593098916398076
--------------------
Undefined classification
HCT
0.6469828467080323
--------------------
Low Weight
Urine
0.4873191634699364
--------------------
Normal Weight
Mg
0.5709812876480205
--------------------
Overweight
MAP
0.526813176022993
--------------------
Obesity 1
Bilirubin
0.48034552174968403
--------------------
Obesity 2
WBC
0.446878399316957
--------------------
Obesity 3
FiO2
0.47203633593304234
--------------------


Maximum MAE value in each subgroup

In [57]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Albumin
0.7463954870157028
--------------------
Female
AST
0.7565732521313827
--------------------
Male
Creatinine
0.7285521064107695
--------------------
Undefined Gender
MechVent
1.0643811764346358
--------------------
+65
BUN
0.7255498613054194
--------------------
-65
DiasABP
0.8156441578600511
--------------------
ICUType 1
HCT
0.8218244252064445
--------------------
ICUType 2
K
0.6633662789027489
--------------------
ICUType 3
TroponinI
0.842596466376899
--------------------
ICUType 4
SysABP
0.7555944034928384
--------------------
Undefined classification
SysABP
0.7544810939173036
--------------------
Low Weight
TroponinI
1.0675889203929847
--------------------
Normal Weight
Temp
0.7669488691837123
--------------------
Overweight
RespRate
0.7106134022293207
--------------------
Obesity 1
HCO3
1.2535198925354494
--------------------
Obesity 2
Na
1.0899931865042596
--------------------
Obesity 3
RespRate
0.961257497137432
--------------------
